In [1]:
import pandas as pd
import numpy as np
from kerastuner import HyperModel, BayesianOptimization
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import kerastuner as kt

import kagglehub
import pandasql
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import TimeSeriesSplit,train_test_split
from category_encoders import TargetEncoder , HelmertEncoder

C:\Users\jayson3xxx\AppData\Local\Temp\ipykernel_22896\3620069876.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, BayesianOptimization


In [2]:
storeDf = pd.read_csv("stores.csv")
holydaysEventsDf = pd.read_csv("holidays_events.csv")
oilDf = pd.read_csv("oil.csv")
transactionsDf = pd.read_csv('transactions.csv')
trainDf = pd.read_csv('train.csv')

In [3]:
TotalDf = pd.merge(oilDf,holydaysEventsDf,on=['date'])
TotalDf = pd.merge(TotalDf,transactionsDf,on=['date'])
TotalDf = pd.merge(TotalDf,storeDf, on=['store_nbr'])
TotalDf = pd.merge(TotalDf,trainDf, on=['date','store_nbr'])
TotalDf.dropna(inplace=True)
TotalDf.drop(columns=['id' , 'description'] , inplace=True)
TotalDf['sales'] = TotalDf.pop('sales')
TotalDf['date'] = pd.to_datetime(TotalDf['date'])
TotalDf.head()

,date,dcoilwtico,type_x,locale,locale_name,transferred,store_nbr,transactions,city,state,type_y,cluster,family,onpromotion,sales
33,2013-02-11,97.01,Holiday,National,Ecuador,False,1,396,Quito,Pichincha,D,13,AUTOMOTIVE,0,0.0
34,2013-02-11,97.01,Holiday,National,Ecuador,False,1,396,Quito,Pichincha,D,13,BABY CARE,0,0.0
35,2013-02-11,97.01,Holiday,National,Ecuador,False,1,396,Quito,Pichincha,D,13,BEAUTY,0,0.0
36,2013-02-11,97.01,Holiday,National,Ecuador,False,1,396,Quito,Pichincha,D,13,BEVERAGES,0,172.0
37,2013-02-11,97.01,Holiday,National,Ecuador,False,1,396,Quito,Pichincha,D,13,BOOKS,0,0.0


In [7]:
data = TotalDf.drop(['date'], axis=1)

# Кодирование категориальных признаков (если они есть)
data = pd.get_dummies(data, columns=['type_x', 'locale', 'locale_name', 'city', 'state', 'type_y', 'family'], drop_first=True)

# Нормализация данных
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
# Создание последовательностей временных данных с дополнительными признаками
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])  # Все столбцы, включая дополнительные признаки
        y.append(data[i + seq_length, -1])  # Целевая переменная 'sales'
    return np.array(X), np.array(y)

SEQ_LENGTH = 7  # Используем 7 дней для прогноза
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# encoder = HelmertEncoder()
# df_encoded = encoder.fit_transform(TotalDf[['type_x', 'locale', 'locale_name', 'city', 'state', 'type_y', 'family']], TotalDf['sales'])


In [9]:
def model_builder(hp):
    model = Sequential()

    # Выбор количества слоев LSTM
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(LSTM(
            units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
            activation=hp.Choice('activation', ['relu', 'tanh']),
            return_sequences=True if i < hp.get('num_layers')-1 else False,
            input_shape=(X_train.shape[1], X_train.shape[2])
        ))

    model.add(Dense(1))

    # Выбор оптимизатора и скорости обучения
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')

    if optimizer == 'adam':
        opt = Adam(learning_rate=lr)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=lr)
    else:
        opt = RMSprop(learning_rate=lr)

    model.compile(
        optimizer=opt,
        loss='mse',
        metrics=['mae']
    )
    return model

# Настройка поиска гиперпараметров
tuner = BayesianOptimization(
    model_builder,
    objective='val_loss',
    max_trials=15,
    executions_per_trial=2,
    directory='keras_tuner_dir',
    project_name='sales_forecast'
)

# Запуск поиска
tuner.search(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
)

# Вывод лучших параметров
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Лучшие гиперпараметры:
- Количество слоев: {best_hps.get('num_layers')}
- Количество нейронов: {[best_hps.get(f'units_{i}') for i in range(best_hps.get('num_layers'))]}
- Активация: {best_hps.get('activation')}
- Оптимизатор: {best_hps.get('optimizer')}
- Скорость обучения: {best_hps.get('learning_rate')}
""")


Trial 15 Complete [00h 06m 43s]
val_loss: 1.774892098183045e-05

Best val_loss So Far: 1.1755855666706339e-07
Total elapsed time: 02h 33m 20s

Лучшие гиперпараметры:
- Количество слоев: 1
- Количество нейронов: [192]
- Активация: tanh
- Оптимизатор: adam
- Скорость обучения: 0.00036448699004144517



In [11]:
model = tuner.get_best_models(num_models=1)[0]
model.save('final_model.h5')

c:\Users\jayson3xxx\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\jayson3xxx\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error as RMSE
prediction = model.predict(X_test)

print(RMSE(y_test , prediction))


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
0.00027098460200560096
